In [ ]:
#work flow for machine learning

#1. 데이터 분류 -> human's work
#2. 연관성 찾기 -> 입력값과 예측값의 연관성을 파악한 후 기계에 넣을 자료 정제
#3. 텍스트 변환 -> 데이터를 숫자로 변환
#4. 빠진값 보완 -> 비어있는 데이터 보완
#5. 차트 만들기 -> 도표를 그려 데이터 한눈에 보기
#6.  모델 제출  -> 만들어진 모델 제출 만들어진 알고리즘을 따라 결과값 확인.

In [ ]:
import pandas as pd 
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt

#머신러닝 툴
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import  GaussianNB
from sklearn.tree import  DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# 세트 두개 불러오기
# 패널 데이터 분석 도구로 불러오기.

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
train.head(5)

In [ ]:
# passenger Id : 승객번호 // sex : 성별 // Age : 나이 // SibSp : 같이 승선한 가족수
# Ticket : 표 이름 ( 단순 이름으로 중요하지 않음 )
# Fare : 요금 -> 비싼 티켓값과의 연관관계 확인해 볼 필요 있음.
# Cabin : 객실 // Pclass : 객실 등급 (Fare 과 연관되어 좋은 객실과 연결 될 것)
# Embarked : 탑승 위치
# train csv 에서는 11개의 column 이 있음.  test 에서는 survived 가 빠진 10개가 있음

In [ ]:
train.info()
print('-'*100)
test.info()

In [ ]:
# 데이터의 양은 훈련 > 테스트 일것
# 훈련세트엔 891개 entry , 12 column 존재
# test 엔 418 개 entry , 11 column 존재
train.isnull().sum()

# 빠진 값 : 나이 177개 , 객실 687개, 탑승지 2개
#자료를 알아보기 위해 컬럼 분류

#손가락으로 셀 수 있는 범위가 나와있는 cartegorical(엑셀처럼 나오니 간단히 파악) / numeric (시각화 자료를 통해 파악해야 함)
# 데이터의 양은 훈련 > 테스트 일것
# 훈련세트엔 891개 entry , 12 column 존재
# test 엔 418 개 entry , 11 column 존재
train.isnull().sum()

# 빠진 값 : 나이 177개 , 객실 687개, 탑승지 2개
#자료를 알아보기 위해 컬럼 분류


In [ ]:
#손가락으로 셀 수 있는 범위가 나와있는 cartegorical(엑셀처럼 나오니 간단히 파악) / numeric (시각화 자료를 통해 파악해야 함)
# describe 설명해줘 기계야!
train.describe()

In [ ]:
# 1. 총 2,224명 중 훈련샘플은 891개 이다 -> 총 데이터 중 40%가 제공됨.
# 2. 살았는지 죽었는지는 1 과 0으로 구분
# 3. 타이타닉 실제 생존률은 32% 이다. -> describe 의 평균 생존률은 38%로 제공됨. -> 실제 데이터가 아닌 훈련용 데이터임.
# 4. Parch 항목 보면 max 0~75% 는 0이다? -> 0 = 혼자탐 -> 혼자 탄 사람이 75% 이상이다.
# 5. SibSp 항목 75% 이상의 값은 1인데 MAX 값에 몰려있다 -> 형제 자매 배우자와 같이 탄 사람. 
# 6. 나이 항목 : 최소 0.42 최대 80 이지만, 평균 28세정도이다 -> 젊은 편
# 7. Pclass : 50% 이상이 3등선실에 타고있다.


In [16]:
# 글자로 된 데이터 보기
# 글자 -> object 포함 include->
train.describe(include='O')

# 동명이인이 없다는 사실을 알 수 있음.
# 21세기에는 여러 성별이 존재하지만 사고 당시에는 딱 2개의 성별.
# top-sex  : male  남자가 더 많이 탔다 // freq 값이 577로 남자가 577명이다.
# Cabin 객실 값을 보자하니, 일반적으로 생각해보면 모든 승객은 1인실을 탈 수 없음. -> 객실 공유는 필수불가결 친구끼리의 객실 이름은 같을 것.
# 즉 대체적으로 승객들은 같은 객실에 탔을 확률이 높았다.
# 승선지는 S 에서 탄 사람이 가장 많다. 644명
# unique row 는 내생각엔 변수의 수라고 보여짐. sex 2개 Embarked 3개 등.
# top 은 최고로 꼽힌 값(최대값) freq 은 top 값의 수 male - 577 // s - 644


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Bryhl, Mr. Kurt Arnold Gottfrid",male,1601,C23 C25 C27,S
freq,1,577,7,4,644


In [ ]:
# 데이터 의미 파악 끝.

#survived 값을 더 정확하게 파악하기 위해 데이터 엔지니어링을 해보고자 함.
